# Import everything

In [ ]:
import csv
import torch
from tqdm.notebook import tqdm
from helper import loader, boilerplate
from settings.cfg import model, device
from settings.cfg import batch_size, tests_paths
from settings.cfg import model_path, result_path

# Get the dataset


In [ ]:
test_image, test_names = loader.load_data(tests_paths, mmap_mode='c')

# Boilerplate Code

In [ ]:
Data = boilerplate.ExportDataset

# Model

In [ ]:
model.load_state_dict(torch.load(model_path))


# Run tests

In [ ]:
tests = Data(test_image, test_names)
tests_dataloader = torch.utils.data.DataLoader(tests, batch_size=batch_size)

with torch.no_grad(), open(result_path, "w", newline='') as result:
    result_writer = csv.writer(result)
    result_writer.writerow(["Frame", "Label"])
    for image, names in tqdm(tests_dataloader):
        image = image.to(device, dtype=torch.float)

        output = model(image)
        output = output.argmax(dim=1).cpu().numpy()
        assert(output.shape[0] == len(names))
        
        result_writer.writerows(zip(names, output))


# Submit


## Create environment variables

In [ ]:
from os import environ
from settings.cfg import kaggle_path
environ['KAGGLE_CONFIG_DIR'] = kaggle_path


In [ ]:
from datetime import datetime
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()


In [ ]:
api.competition_submit(
    result_path, f"Submitted at {datetime.now()}", "hsgs-hackathon2022")
